## Setup

In [ ]:
!pip install poetry

In [ ]:
!pip install https://github.com/swanandlab/SMorph/releases/download/v0.1.1-alpha/SMorph-0.1.1.tar.gz

In [3]:
on_colab = 'google.colab' in str(get_ipython())

import warnings
warnings.filterwarnings('ignore')
if not on_colab:
    import napari
import smorph.util.autocrop as ac

In [4]:
# Helper function
def view_3D(*args):
    """Views a 3D image."""
    n_images = len(args)
    if not on_colab and n_images > 0:  
        with napari.gui_qt():
            viewer = napari.view_image(**args[0], ndisplay=3)
            for itr in range(1, n_images):
                if args[itr]['data'].max() > 1 and args[itr]['data'].max() % 1 == 0:
                    if 'colormap' in args[itr].keys():
                        del args[itr]['colormap']
                    if 'gamma' in args[itr].keys():
                        del args[itr]['gamma']
                    viewer.add_labels(**args[itr])
                else:
                    viewer.add_image(**args[itr])

In [5]:
SOMA_SELECTED = 'Autocropped/21_DAY_CA1SR - Copy/DMI_MSP2.1MA_3_LONG MARK_20X_SEC 2_LEFT CA1sr_21 DAY-CA1SR/residue'

## Select somas of individual cells in clumps

### Option 1. Pre-selected point ROIs

In [ ]:
reconstructed_labels = None
reconstructed_labels, parent_path, roi_path = ac.postprocess_segment(SOMA_SELECTED, reconstructed_labels)

In [ ]:
view_3D({'data': reconstructed_labels, 'colormap': 'inferno', 'name': 'reconstructed_labels'})

### Option 2. Manually-select point ROIs

In [6]:
reconstructed_seg = reconstructed_labels = None
reconstructed_seg, parent_path, roi_path, somas_est = ac.manual_postprocess(SOMA_SELECTED, reconstructed_seg)

In [16]:
if not on_colab:
    with napari.gui_qt():
        man_viewer = napari.view_image(reconstructed_seg)
        man_viewer.add_points(somas_est, face_color='red', size=5, symbol='cross')

Exception in callback BaseAsyncIOLoop._handle_events(40676, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(40676, 1)>
Traceback (most recent call last):
  File "c:\users\swanandlab\appdata\local\programs\python\python37\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\SwanandLab\AppData\Local\pypoetry\Cache\virtualenvs\smorph-ObnBYCc1-py3.7\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "C:\Users\SwanandLab\AppData\Local\pypoetry\Cache\virtualenvs\smorph-ObnBYCc1-py3.7\lib\site-packages\zmq\eventloop\zmqstream.py", line 448, in _handle_events
    self._handle_recv()
  File "C:\Users\SwanandLab\AppData\Local\pypoetry\Cache\virtualenvs\smorph-ObnBYCc1-py3.7\lib\site-packages\zmq\eventloop\zmqstream.py", line 477, in _handle_recv
    self._run_callback(callback, msg)
  File "C:\Users\SwanandLab\AppData\Local\pypoetry\Cache\virtualenvs\smorph-ObnBYCc1-py3.7\l

In [17]:
import json
from os import listdir, path

import numpy as np
import tifffile
from skimage.measure import label
from smorph.util.autocrop._io import _build_multipoint_roi

somas_est = np.unique(man_viewer.layers[1].data, axis=0)

for file in listdir(SOMA_SELECTED):
    if not file.startswith('.') and file.endswith(('.tif', '.tiff')) and '_mip' not in file:
        name = SOMA_SELECTED + '/' + file
        image = tifffile.TiffFile(name)
        metadata = image.pages[0].tags['ImageDescription'].value
        metadata = json.loads(metadata)

        try :
            im = image.asarray()

            if reconstructed_labels is None:
                parent = ac._io.import_confocal_image(parent_path)
                reconstructed_labels = np.zeros(parent.shape)

            minz, miny, minx, maxz, maxy, maxx = metadata['bounds']
            linebuilder = ac._roi_extract._load_ROI(roi_path)
            X, Y = ac.core._unwrap_polygon(linebuilder)
            min_x, max_x = int(min(X)), int(max(X) + 1)
            min_y, max_y = int(min(Y)), int(max(Y) + 1)
            miny += min_y
            maxy += min_y
            minx += min_x
            maxx += min_x
            ll = np.array([minz, miny, minx])  # lower-left
            ur = np.array([maxz, maxy, maxx])  # upper-right
            mask_coords = np.array(np.where(im>0)).T
            inidx = np.all(np.logical_and(ll <= somas_est, somas_est <= ur), axis=1)

            somas_coords = somas_est[inidx]
            somas_coords -= np.array([minz+1, miny+1, minx+1])
            
            masked_somas_coords = []
            for soma_coords in somas_coords:
                if soma_coords in mask_coords:
                    masked_somas_coords.append(soma_coords.astype(int))

            markers = np.zeros(im.shape)
            for i in range(1, len(masked_somas_coords) + 1):
                markers[tuple(map(int, masked_somas_coords[i-1]))] = i

            soma_roi_path = '.'.join(name.split('.')[:-1]) + '.roi'
            if len(masked_somas_coords):  # and not path.isfile(soma_roi_path)
                _build_multipoint_roi(np.array(masked_somas_coords) + 1 + np.array([1, 0, 0])).tofile(soma_roi_path)

            labels = ac._postprocessing._segment_clump(im, markers)
            reconstructed_labels[minz:maxz, miny:maxy, minx:maxx] += labels
        except Exception as e:
            print(e)
reconstructed_labels = label(reconstructed_labels)

In [18]:
view_3D({'data': reconstructed_seg, 'colormap': 'inferno', 'name': 'reconstructed_seg'},
        {'data': reconstructed_labels, 'colormap': 'inferno', 'name': 'reconstructed_labels'})

Exception in callback BaseAsyncIOLoop._handle_events(51496, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(51496, 1)>
Traceback (most recent call last):
  File "c:\users\swanandlab\appdata\local\programs\python\python37\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\SwanandLab\AppData\Local\pypoetry\Cache\virtualenvs\smorph-ObnBYCc1-py3.7\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "C:\Users\SwanandLab\AppData\Local\pypoetry\Cache\virtualenvs\smorph-ObnBYCc1-py3.7\lib\site-packages\zmq\eventloop\zmqstream.py", line 448, in _handle_events
    self._handle_recv()
  File "C:\Users\SwanandLab\AppData\Local\pypoetry\Cache\virtualenvs\smorph-ObnBYCc1-py3.7\lib\site-packages\zmq\eventloop\zmqstream.py", line 477, in _handle_recv
    self._run_callback(callback, msg)
  File "C:\Users\SwanandLab\AppData\Local\pypoetry\Cache\virtualenvs\smorph-ObnBYCc1-py3.7\l

In [19]:
reconstructed_volume = (reconstructed_labels > 0).sum()
f'Postreconstructing Volume: {reconstructed_volume}'

'Postreconstructing Volume: 176359'

In [20]:
linebuilder = ac._roi_extract._load_ROI(roi_path)

In [21]:
import numpy as np
from skimage.morphology import binary_erosion
from skimage.segmentation import clear_border
from skimage.draw import polygon2mask

conservative = True
reconstructed_residue_labels = clear_border(reconstructed_labels, mask=None if conservative
                        else ac.core._compute_convex_hull(thresholded))

if linebuilder is not None:
    X, Y = ac.core._unwrap_polygon(linebuilder)
    min_x, max_x = int(min(X)), int(max(X) + 1)
    min_y, max_y = int(min(Y)), int(max(Y) + 1)
    shape = reconstructed_labels.shape
    roi_mask = np.empty(shape)
    roi_mask[0] = polygon2mask(shape[1:][::-1], list(zip(X, Y))).T

    for i in range(1, shape[0]):
        roi_mask[i] = roi_mask[0]
    roi_mask = binary_erosion(roi_mask)
    reconstructed_residue_labels = clear_border(reconstructed_residue_labels, mask=roi_mask)

reconstructed_filtered_regions = ac.arrange_regions(reconstructed_residue_labels)

In [ ]:
view_3D({'data': reconstructed_residue_labels, 'colormap': 'gray', 'name': 'reconstructed_filtered_regions'},
        {'data': reconstructed_labels, 'gamma': .8, 'colormap': 'inferno', 'name': 'reconstructed_labels'},
        {'data': roi_mask, 'gamma': .8, 'colormap': 'inferno', 'name': 'roi_mask'})

In [22]:
X, Y = ac.core._unwrap_polygon(linebuilder)
min_x, max_x = int(min(X)), int(max(X) + 1)
min_y, max_y = int(min(Y)), int(max(Y) + 1)

In [23]:
reconstructed_residue_labels = ac.filter_labels(reconstructed_labels[:, min_y:max_y, min_x:max_x], None, linebuilder, True)

In [24]:
reconstructed_residue_volume = (reconstructed_residue_labels > 0).sum()
f'Postreconstructing-residue Volume: {reconstructed_residue_volume}; ' \
f'%age volume: {(reconstructed_residue_volume) / reconstructed_volume * 100} %'

'Postreconstructing-residue Volume: 100682; %age volume: 57.08923275818075 %'

In [ ]:
view_3D(#{'data': denoised, 'colormap': 'inferno', 'name': 'denoised'},
        {'data': reconstructed_residue_labels, 'colormap': 'gray', 'gamma': .8, 'name': 'reconstructed_filtered_labels'})
        #{'data': filtered_labels, 'colormap': 'inferno', 'gamma': .8, 'name': 'filtered_labels'},
        #{'data': labels, 'colormap': 'gist_earth', 'gamma': .8, 'name': 'labels'})

In [25]:
reconstructed_filtered_regions = ac.arrange_regions(reconstructed_residue_labels)

original = ac.import_confocal_image(parent_path)

if original.ndim == 2:
  original = (original - original.min()) / (original.max() - original.min())
  import numpy as np
  original = np.expand_dims(original, 0)

LOW_VOLUME_CUTOFF = 0  # filter noise/artifacts
HIGH_VOLUME_CUTOFF = 1e9  # filter cell clusters

OUTPUT_OPTION = 'both'  # '3d' for 3D cells, 'mip' for Max Intensity Projections
SEGMENT_TYPE = 'segmented'  # unsegmented output erroneous (cropped different region)

NAME_ROI = 'CA1SR'

ac.export_cells(parent_path, LOW_VOLUME_CUTOFF,
                HIGH_VOLUME_CUTOFF, OUTPUT_OPTION, original,
                reconstructed_filtered_regions, None, SEGMENT_TYPE, NAME_ROI, linebuilder, roi_path=roi_path)

ZeroDivisionError: integer division or modulo by zero